<img src="https://drive.google.com/uc?export=view&id=15SNbs9etrW7wdww8q0BYpewEgOKC1d3m" width="100%" alt="Encabezado unidad 2"></img>

# Conceptos de Cassandra
---

En este notebook mostraremos algunas generalidades y formas de uso de la base de datos _Cassandra_ y su lenguaje de consulta CQL.

Primero instalaremos e importaremos las librerías Necesarias:

In [ ]:
!pip install cassandra-driver

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
from IPython import display

Ahora, realizaremos la conexión a la base de datos de [astra](https://astra.datastax.com/) tal y cómo se mostró en el notebook `M3U2NB1_cassandra_python.ipynb`:

> **Nota**: recuerde subir los archivos `bundle.zip` y `token.json`.

In [ ]:
cloud_config= {
        "secure_connect_bundle": "bundle.zip" # nombre de su archivo de bundle
}
with open('token.json') as user_file:
  token = json.loads(user_file.read())

In [ ]:
auth_provider = PlainTextAuthProvider(
        token["clientId"],
        token["secret"]
        )
cluster = Cluster(
        cloud=cloud_config,
        auth_provider=auth_provider
        )
session = cluster.connect()

Cassandra maneja un lenguaje de consulta llamado **Cassandra Query Languaje** (CQL), el cual se caracteriza por ser muy cercano a SQL, en especial a _MySQL_. En especial la jerarquía de organización de datos tiene la siguiente analogía:

| SQL | Cassandra |
| --- | --- |
| DATABASE | KEYSPACE |
| TABLE | TABLE |

Aunque _CQL_ resulta ser muy cercano a _SQL_, el enfoque es totalmente diferente y hay varios conceptos que hay que considerar para el diseño del modelo de datos y las tablas, en especial _Cassandra_ y varias bases de datos NoSQL siguen un enfoque de **Query First Design**.

En este caso comenzaremos mostrando la forma en la que debemos diseñar tablas en _Cassandra_ y algunas particularidades de esta base de datos:

## **1. Query First Design**
---

A diferencia de otros modelos para la estructuración de tablas y bases de datos, _Cassandra_ sigue un enfoque de diseño enfocado a la consulta, es decir, debemos construir nuestras tablas de una forma en la que la consulta sea bastante simple.

Esto involucra las siguientes consideraciones:

* La tabla debe estar construida para que los filtros sean simples, es decir, como máximo filtramos usando operadores de igualdad sobre un campo y unimos condiciones sólo con `AND`.
* Es mejor tener los resultados pre-calculados aunque sean redundantes. Esto se hace notorio ante la falta de la operación `JOIN` en _Cassandra_, ya que, es mejor tener una tabla con el resultado del cruce de datos y no tener que cruzar estos datos cuando se ejecute la consulta.
* El motor de _Cassandra_ al estar enfocado en la consulta, también está pensado para que las operaciones de consulta sean rápidas. _Cassandra_ permite ejecutar consultas de forma distribuida y eficiente.

Veamos un ejemplo de un modelo de datos clásico en SQL para datos de estudiantes de un colegio y sus acudientes:

* **students**

| id | name | age | gender | grade |
|---|---|---|---|---|
| 1 | Bart Simpson | 10 | male | 3.0 |
| 2 | Lisa Simpson | 8 | female | 4.9 |
| 3 | Milhouse Van Houten | 11 | male | 2.8 |

* **parents**

| id | name | age | gender | student_id |
|---|---|---|---|---|
| 1 | Homero Simpson | 39 | male | 1 |
| 2 | Marge Simpson | 33 | female | 1 |
| 3 | Homero Simpson | 50 | male | 2 |
| 4 | Marge Simpson | 50 | female | 2 |
| 5 | Kirk Van Houten | 40 | male | 3 |
| 6 | Luann Van Houten |  32 | female | 3 |

Si quisieramos encontrar la lista de estudiantes y sus acudientes desde _SQL_, probablemente construiríamos una consulta de la siguiente forma:

```sql
SELECT
    students.name AS student_name,
    parents.name AS parent_name
FROM
    students
INNER JOIN
    parents
ON
    students.id = parents.student_id
;
```

Este comportamiento no es posible en Cassandra, para ello tendríamos que tener una tabla con el inner join definida desde un inicio:

* **students_parents**

| student_name | parent_name |
| --- | --- |
| Bart Simpson | Homero Simpson |
| Bart Simpson | Marge Simpson |
| Lisa Simpson | Homero Simpson |
| Lisa Simpson | Marge Simpson |
| Milhouse Van Houten | Kirk Van Houten |
| Milhouse Van Houten | Luann Van Houten |

De esta forma, en _Cassandra_ se extraería la información de la siguiente forma:

```sql
SELECT
    *
FROM
    students_parents
;
```

Note la simplicidad en la consulta, lo que es el objetivo final en _Cassandra_.

Veamos algunos métodos para la manipulación de datos en _Cassandra_:

## **2. Creación**
---

La sintaxis para la creación de tablas en _Cassandra_ es la siguiente:

```sql
CREATE TABLE [IF NOT EXISTS] <nombre_keyspace>.<nombre_tabla> (
    nombre_columna1 TIPO1,
    nombre_columna2 TIPO2,
    nombre_columna3 TIPO3
    PRIMARY KEY(nombre_columna1, nombre_columna2)
    )
;
```

Veamos un ejemplo:

In [ ]:
query = """
CREATE TABLE IF NOT EXISTS test.students (
        id INT PRIMARY KEY,
        name TEXT,
        age INT,
        gender TEXT,
        grade FLOAT
        )
;
"""
session.execute(query)

Puede ver información de las tablas accediendo al objeto `cluster`:

In [ ]:
cluster.metadata.keyspaces

In [ ]:
cluster.metadata.keyspaces["test"].tables

In [ ]:
cluster.metadata.keyspaces["test"].tables["students"].columns

Ahora, veamos un ejemplo de inserción de datos. La sintaxis es la siguiente:

```sql
INSERT INTO <nombre_keyspace>.<nombre_tabla> (
    nombre_columna1, nombre_columna2
    )
VALUES
    (<valor1>, <valor2>)
;
```

Veamos un ejemplo

In [ ]:
query = """
INSERT INTO test.students (
        id, name, age, gender, grade
        )
VALUES
    (1, 'Bart Simpson', 10, 'male', 3.0)
;
"""
session.execute(query)

También es posible insertar multiples valores al tiempo (batch), de la siguiente forma:

In [ ]:
query = """
BEGIN BATCH
    INSERT INTO test.students (id, name, age) VALUES (2, 'Lisa Simpson', 8);
    INSERT INTO test.students (id, name, gender) VALUES (3, 'Milhouse Van Houten', 'male');
APPLY BATCH
;
"""
session.execute(query)

En _Cassandra_ hay un concepto crucial que hay que considerar para la creación de las tablas. Se trata de los tres tipos de llaves que tiene:

* **Llaves compuestas**: se trata de llaves que se conforman por varias columnas, en especial una llave primaria es una llave compuesta por llaves de partición y llaves de agrupamiento.
* **Llaves de partición**: son llaves que _Cassandra_ utiliza para la indexación, y como su nombre lo indica son usadas para determinar en qué partición física (nodo de almacenamiento o token) se ubicarán los datos.
* **Llaves de agrupamiento**: son llaves que _Cassandra_ utiliza para permitir filtros rápidos por grupos y operaciones de agregación `GROUP BY`. Internamente, _Cassandra_ utiliza estas llaves para determinar el orden en el que se depositan los datos dentro de la partición.

Generalmente, estas llaves se definen de la siguiente forma dentro de la operación de creación de tablas:

```sql
PRIMARY KEY( (columna1, columna2), columna3, columna4 )
```

En este caso, `columna1` y `columna2` son llaves de partición y `columna3` y `columna4` son llaves de agrupamiento.

> **Nota**: puede usar multiples llaves de partición o de agrupamiento.

Veamos la creación de una tabla con llaves de partición y de agrupamiento:

In [ ]:
query = """
CREATE TABLE test.students2 (
        id UUID,
        name TEXT,
        age INT,
        gender TEXT,
        grade FLOAT,
        PRIMARY KEY ((id, name), gender)
        )
;
"""
session.execute(query)

En este caso, `id` y `name` son llaves de partición y `gender` es una llave de agrupamiento.

El tipo `UUID` es comunmente usado en lugar de valores autoincrementales como en _SQL_, esto se hace principalmente por que _Cassandra_ tiene datos particionados (en distintas ubicaciones) y tener un orden no tiene mucho sentido.

## **3. Consulta**
---

En _Cassandra_ las operaciones de consulta suelen ser más simples, el éxito de una implementación de una base de datos en _Cassandra_ depende fundamentalmente de un buen diseño de tablas (query first) y una buena selección de llaves primarias y de partición.

Veamos algunos ejemplos de consultas y qué operaciones se pueden hacer sobre distintos tipos de llaves, primero vamos a crear una tabla y a llenarla con algunos datos

In [ ]:
query = """
CREATE TABLE test.only_partition (
        id INT PRIMARY KEY,
        category TEXT,
        )
;
"""
session.execute(query)

In [ ]:
query = """
BEGIN BATCH
INSERT INTO test.only_partition (id, category) VALUES (1, 'A');
INSERT INTO test.only_partition (id, category) VALUES (2, 'B');
INSERT INTO test.only_partition (id, category) VALUES (3, 'B');
APPLY BATCH;
"""
session.execute(query)

Comenzamos con una selección completa de los datos de la tabla, usamos el método `all()` para extraer el resultado:

In [ ]:
query = """
SELECT * FROM test.only_partition;
"""
session.execute(query).all()

Ahora, vamos a intentar filtrar por el campo `id`:

In [ ]:
query = """
SELECT
    *
FROM
    test.only_partition
WHERE
    id = 1
;
"""
try:
    print(session.execute(query).all())
except Exception as e:
    display(e)

Ahora, vamos a intentar filtros con desigualdades:

In [ ]:
query = """
SELECT
    *
FROM
    test.only_partition
WHERE
    id > 1
;
"""
try:
    print(session.execute(query).all())
except Exception as e:
    display(e)

Note que esta consulta **genera error**, esto se debe a que las desigualdades por lo general requieren la comparación de múltiples valores y esto resulta ser muy costoso con tablas muy grandes.

Ahora, vamos a filtrar con respecto al otro campo:

In [ ]:
query = """
SELECT
    *
FROM
    test.only_partition
WHERE
    category = 'A'
;
"""
try:
    print(session.execute(query).all())
except Exception as e:
    display(e)

Esta consulta también **genera error**, y esto se debe a que estamos tratando de filtrar con un campo no indexable. En _Cassandra_ se recomienda filtrar unicamente sobre las llaves de partición y las llaves de agrupamiento por motivos de velocidad en la consulta.

Veamos un ejemplo con una tabla de múltiples llaves de partición y de agrupamiento:

In [ ]:
query = """
CREATE TABLE
    test.multiple_keys (
        name TEXT,
        last_name TEXT,
        category TEXT,
        sub_category TEXT,
        age INT,
        PRIMARY KEY ((name, last_name), category, sub_category)
        )
;
"""
session.execute(query)

En este caso tenemos `name` y `last_name` como llaves primarias, `category` y `sub_category` son llaves de agrupamiento.

Vamos a insertar algunos valores:

In [ ]:
query = """
BEGIN BATCH
    INSERT INTO
        test.multiple_keys (
            name, last_name, category, sub_category, age
            )
    VALUES
        ('Pedro', 'Picapiedra', 'A', 'casado', 50)
    ;
    INSERT INTO
        test.multiple_keys (
            name, last_name, category, sub_category, age
            )
    VALUES
        ('Pedro', 'Montenegro', 'A', 'soltero', 49)
    ;
    INSERT INTO
        test.multiple_keys (
            name, last_name, category, sub_category, age
            )
    VALUES
        ('Homero', 'Simpson', 'B', 'casado', 45)
    ;
    INSERT INTO
        test.multiple_keys (
            name, last_name, category, sub_category, age
            )
    VALUES
        ('Anapolo', 'Rocondo', 'B', 'soltero', 45)
    ;
APPLY BATCH;
"""
session.execute(query)

Veamos una consulta donde filtramos por el campo `name`:

In [ ]:
query = """
SELECT
    *
FROM
    test.multiple_keys
WHERE
    name = 'Anapolo'
;
"""
try:
    print(session.execute(query).all())
except Exception as e:
    display(e)

Note que esta consulta **genera error** a pesar que `name` es una llave de partición. Esto se debe a que las consultas en _Cassandra_ están pensadas para realizarse sobre todas las llaves de partición al tiempo y de forma obligatoria:

In [ ]:
query = """
SELECT
    *
FROM
    test.multiple_keys
WHERE
    name = 'Anapolo' AND last_name = 'Rocondo'
;
"""
try:
    print(session.execute(query).all())
except Exception as e:
    display(e)

Los filtros sobre las llaves de agrupamiento son opcionales, veamos un ejemplo, con una nueva tabla:

In [ ]:
query = """
CREATE TABLE
    test.clustering_keys (
        name TEXT,
        category TEXT,
        sub_category TEXT,
        age INT,
        PRIMARY KEY (name, category, sub_category)
        )
;
"""
session.execute(query)

Insertamos algunos valores:

In [ ]:
query = """
BEGIN BATCH
    INSERT INTO
        test.clustering_keys (
            name, category, sub_category, age
            )
    VALUES
        ('Pedro', 'A', 'casado', 50)
    ;
    INSERT INTO
        test.clustering_keys (
            name, category, sub_category, age
            )
    VALUES
        ('Pedro', 'A', 'soltero', 49)
    ;
    INSERT INTO
        test.clustering_keys (
            name, category, sub_category, age
            )
    VALUES
        ('Homero', 'B', 'casado', 45)
    ;
    INSERT INTO
        test.clustering_keys (
            name, category, sub_category, age
            )
    VALUES
        ('Homero', 'B', 'soltero', 45)
    ;
APPLY BATCH;
"""
session.execute(query)

Veamos filtros sobre columnas de agrupamiento:

In [ ]:
query = """
SELECT
    *
FROM
    test.clustering_keys
WHERE
    name = 'Pedro'
;
"""
try:
    print(session.execute(query).all())
except Exception as e:
    display(e)

In [ ]:
query = """
SELECT
    *
FROM
    test.clustering_keys
WHERE
    name = 'Pedro' AND
    category = 'A'
;
"""
try:
    print(session.execute(query).all())
except Exception as e:
    display(e)

Como puede ver, las llaves de agrupamiento son opcionales, aunque son útiles para agrupar:

In [ ]:
query = """
SELECT
    category,
    AVG(age) AS age_avg
FROM
    test.clustering_keys
GROUP BY
    name, category
;
"""
try:
    print(session.execute(query).all())
except Exception as e:
    display(e)

## **4. Actualización**
---

Las operaciones de actualización son similares a las de `SQL`, por ejemplo para modificar columnas usamos comandos `ALTER`, por ejemplo:

In [ ]:
query = """
ALTER TABLE
    test.clustering_keys
ADD
    nueva_columna INT
;
"""
session.execute(query)

In [ ]:
cluster.metadata.keyspaces["test"].tables["clustering_keys"].columns

Para modificar valores podemos usar el comando `UPDATE` como en _SQL_, aunque es necesario especificar todas las llaves de partición y de agrupamiento para la operación:

In [ ]:
query = """
UPDATE
    test.clustering_keys
SET
    age = -1
WHERE
    name = 'Homero' AND category = 'B' and sub_category = 'soltero'
;
"""
session.execute(query)

Veamos cómo queda la tabla:

In [ ]:
query = "SELECT * FROM test.clustering_keys;"
session.execute(query).all()

## **5. Borrado**
---

Para eliminar registros debemos usar la sintaxis `DELETE` como en _SQL_:

In [ ]:
query = """
DELETE FROM
    test.clustering_keys
WHERE
    name = 'Pedro'
;
"""
session.execute(query)

Veamos cómo queda la tabla:

In [ ]:
query = "SELECT * FROM test.clustering_keys;"
session.execute(query).all()

También podemos eliminar columnas, siempre y cuando no sean llaves de agrupamiento ni de partición.

In [ ]:
query = """
ALTER TABLE
    test.clustering_keys
DROP
    age
;
"""
session.execute(query)

In [ ]:
cluster.metadata.keyspaces["test"].tables["clustering_keys"].columns

Finalmente, para eliminar tablas, usamos el comando `DROP` como en _SQL_:

In [ ]:
query = """
DROP TABLE test.students;
"""
session.execute(query, timeout=None)

In [ ]:
query = """
DROP TABLE test.students2;
"""
session.execute(query, timeout=None)

In [ ]:
query = """
DROP TABLE test.only_partition;
"""
session.execute(query, timeout=None)

In [ ]:
query = """
DROP TABLE test.multiple_keys;
"""
session.execute(query, timeout=None)

In [ ]:
query = """
DROP TABLE test.clustering_keys;
"""
session.execute(query, timeout=None)

## **6. Recursos Adicionales**
---

* [Apache Cassandra](https://cassandra.apache.org/_/index.html).
* [CQL reference](https://docs.datastax.com/en/cql-oss/3.3/cql/cql_reference/cqlReferenceTOC.html)
* [Begginers guide to learn cassandra](https://medium.com/@aymannaitcherif/beginners-guide-to-learn-cassandra-part-2-4e8511a4838f)

## **7. Créditos**
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Diseño, desarrollo del notebook y material audiovisual**

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*